In [1]:
import pandas as pd

nhefs = pd.read_csv('../data/nhefs.csv')

In [2]:
nhefs.head()

,seqn,qsmk,death,yrdth,modth,dadth,sbp,dbp,sex,age,...,birthcontrol,pregnancies,cholesterol,hightax82,price71,price82,tax71,tax82,price71_82,tax71_82
0,233,0,0,NaN,NaN,NaN,175.0,96.0,0,42,...,2,NaN,197.0,0.0,2.183594,1.739990,1.102295,0.461975,0.443787,0.640381
1,235,0,0,NaN,NaN,NaN,123.0,80.0,0,36,...,2,NaN,301.0,0.0,2.346680,1.797363,1.364990,0.571899,0.549316,0.792969
2,244,0,0,NaN,NaN,NaN,115.0,75.0,1,56,...,0,2.0,157.0,0.0,1.569580,1.513428,0.551270,0.230988,0.056198,0.320251
3,245,0,1,85.0,2.0,14.0,148.0,78.0,0,68,...,2,NaN,174.0,0.0,1.506592,1.451904,0.524902,0.219971,0.054794,0.304993
4,252,0,0,NaN,NaN,NaN,118.0,77.0,0,40,...,2,NaN,216.0,0.0,2.346680,1.797363,1.364990,0.571899,0.549316,0.792969


In [3]:
nhefs['university'] = nhefs.school >= 16

In [4]:
nhefs['older'] = nhefs.age > 50

In [5]:
nhefs['match'] = 1

In [6]:
nhefs['cens'] = nhefs.wt82.isnull()

In [8]:
nhefs = nhefs[nhefs.education.isnull() == False]

In [9]:
nhefs_nmv = nhefs[nhefs.wt82.isnull() == False]

In [20]:
print nhefs_nmv[nhefs_nmv.qsmk == 1].wt82_71.mean()
print nhefs_nmv[nhefs_nmv.qsmk == 1].wt82_71.std()

4.525078989702234
8.748261237678289


In [21]:
print nhefs_nmv[nhefs_nmv.qsmk == 0].wt82_71.mean()
print nhefs_nmv[nhefs_nmv.qsmk == 0].wt82_71.std()

1.9844975347463456
7.449075855798698


In [24]:
pd.pivot_table(nhefs_nmv, ['age', 'wt71', 'smokeintensity', 'smokeyrs'], columns=['qsmk'])

qsmk,0,1
age,42.788478,46.173697
smokeintensity,21.191745,18.602978
smokeyrs,24.087704,26.032258
wt71,70.302837,72.354888


In [28]:
pd.pivot_table(nhefs_nmv, ['sex', 'race', 'university', 'exercise', 'active', 'older'], columns=['qsmk'])


qsmk,0,1
active,0.631986,0.689826
exercise,1.175408,1.250620
older,0.268272,0.387097
race,0.146174,0.089330
sex,0.533964,0.454094
university,0.098882,0.153846


# Program 12.2

In [72]:
import statsmodels.formula.api as sm

In [ ]:
sm.Logit(nhefs_nmv['qsmk'], nhefs_nmv[['']])

In [59]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))


m = ModelDesc([LookupFactor('qsmk')], model)
m.describe()

'qsmk ~ sex + race + age + I(age ** 2) + education + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + exercise + active + wt71 + I(wt71 ** 2)'

In [61]:
fit_model = sm.Logit.from_formula(m.describe(), nhefs_nmv).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.536432
         Iterations 6
                            Results: Logit
Model:                Logit              No. Iterations:     6.0000   
Dependent Variable:   qsmk               Pseudo R-squared:   0.059    
Date:                 2018-01-20 14:13   AIC:                1708.1036
No. Observations:     1566               BIC:                1783.0915
Df Model:             13                 Log-Likelihood:     -840.05  
Df Residuals:         1552               LL-Null:            -893.03  
Converged:            1.0000             Scale:              1.0000   
----------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------
Intercept              -2.3598   1.3791 -1.7111 0.0871 -5.0628  0.3432
sex                    -0.5560   0.1515 -3.6694 0.0002 -0.8530 -0.2590
rac

In [65]:
nhefs_w = fit_model.predict()

In [69]:
nhefs_w[nhefs_nmv.qsmk == False] = 1 - nhefs_w[nhefs_nmv.qsmk == False]

In [70]:
nhefs_w = 1 / nhefs_w

In [71]:
nhefs_w

array([1.1004459 , 1.18479958, 1.15240257, ..., 1.14164084, 1.61583301,
       5.96330308])

In [79]:
gee = sm.GEE.from_formula('wt82_71 ~ qsmk', groups='seqn', data=nhefs_nmv, weights=nhefs_w).fit()
print gee.summary()

                               GEE Regression Results                              
Dep. Variable:                     wt82_71   No. Observations:                 1566
Model:                                 GEE   No. clusters:                     1566
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                           Gaussian   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 20 Jan 2018   Scale:                          65.374
Covariance type:                    robust   Time:                         14:28:44
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7828      0.225      7.941      0.000       1.343       2.223
qsmk   

In [104]:
import numpy as np
    

nhefs_nmv['ip_w'] = nhefs_w


pd.pivot_table(nhefs_nmv, ['ip_w'], index=['sex'], columns=['qsmk'], aggfunc=np.sum)

/Users/nate/Desktop/causalinferencebook/env/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


ip_w            
qsmk           0           1
sex                         
0     763.230683  760.320235
1     801.846565  799.399224

In [89]:
pd.pivot_table(nhefs_nmv[(nhefs_nmv.sex == 1) & (nhefs_nmv.race == 0)], ['age'], columns=['qsmk'])

qsmk,0,1
age,42.416031,45.536585


# 12.3

In [90]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))


m = ModelDesc([LookupFactor('qsmk')], model)
m.describe()

'qsmk ~ sex + race + age + I(age ** 2) + education + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + exercise + active + wt71 + I(wt71 ** 2)'

In [91]:
fit_model = sm.Logit.from_formula(m.describe(), nhefs_nmv).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.536432
         Iterations 6
                            Results: Logit
Model:                Logit              No. Iterations:     6.0000   
Dependent Variable:   qsmk               Pseudo R-squared:   0.059    
Date:                 2018-01-20 15:01   AIC:                1708.1036
No. Observations:     1566               BIC:                1783.0915
Df Model:             13                 Log-Likelihood:     -840.05  
Df Residuals:         1552               LL-Null:            -893.03  
Converged:            1.0000             Scale:              1.0000   
----------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------
Intercept              -2.3598   1.3791 -1.7111 0.0871 -5.0628  0.3432
sex                    -0.5560   0.1515 -3.6694 0.0002 -0.8530 -0.2590
rac

In [92]:
est_prob_d = fit_model.predict()

In [93]:
fit_model = sm.Logit.from_formula('qsmk ~ 1', nhefs_nmv).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.570260
         Iterations 5
                         Results: Logit
Model:              Logit            No. Iterations:   5.0000   
Dependent Variable: qsmk             Pseudo R-squared: 0.000    
Date:               2018-01-20 15:03 AIC:              1788.0542
No. Observations:   1566             BIC:              1793.4105
Df Model:           0                Log-Likelihood:   -893.03  
Df Residuals:       1565             LL-Null:          -893.03  
Converged:          1.0000           Scale:            1.0000   
-----------------------------------------------------------------
             Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
-----------------------------------------------------------------
Intercept   -1.0598    0.0578  -18.3349  0.0000  -1.1731  -0.9465



In [94]:
est_prob_n = fit_model.predict()

In [95]:
est_prob_d[nhefs_nmv.qsmk == False] = 1 - est_prob_d[nhefs_nmv.qsmk == False]
est_prob_n[nhefs_nmv.qsmk == False] = 1 - est_prob_n[nhefs_nmv.qsmk == False]

nhefs_sw = est_prob_n / est_prob_d

In [97]:
gee = sm.GEE.from_formula('wt82_71 ~ qsmk', groups='seqn', data=nhefs_nmv, weights=nhefs_sw).fit()
print gee.summary()

                               GEE Regression Results                              
Dep. Variable:                     wt82_71   No. Observations:                 1566
Model:                                 GEE   No. clusters:                     1566
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                           Gaussian   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 20 Jan 2018   Scale:                          60.902
Covariance type:                    robust   Time:                         15:06:47
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7828      0.225      7.941      0.000       1.343       2.223
qsmk   

In [103]:
import numpy as np
    

nhefs_nmv['s_ip_w'] = nhefs_sw


pd.pivot_table(nhefs_nmv, ['s_ip_w'], index=['sex'], columns=['qsmk'], aggfunc=np.sum)

/Users/nate/Desktop/causalinferencebook/env/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


s_ip_w            
qsmk           0           1
sex                         
0     566.818189  195.663509
1     595.496523  205.720234

# 12.4

In [105]:
nhefs_nmv_s = nhefs_nmv[nhefs_nmv.smokeintensity <= 25]

In [161]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))


m = ModelDesc([LookupFactor('smkintensity82_71')], model)
m.describe()

'smkintensity82_71 ~ C(sex) + C(race) + age + I(age ** 2) + C(education) + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + C(exercise) + C(active) + wt71 + I(wt71 ** 2)'

In [162]:
fit_model = sm.OLS.from_formula(m.describe(), nhefs_nmv_s).fit()

print fit_model.summary2()

                    Results: Ordinary least squares
Model:               OLS                Adj. R-squared:      0.064    
Dependent Variable:  smkintensity82_71  AIC:                 8697.3746
Date:                2018-01-20 15:53   BIC:                 8793.4747
No. Observations:    1162               Log-Likelihood:      -4329.7  
Df Model:            18                 F-statistic:         5.392    
Df Residuals:        1143               Prob (F-statistic):  2.98e-12 
R-squared:           0.078              Scale:               102.59   
----------------------------------------------------------------------
                        Coef.  Std.Err.    t    P>|t|   [0.025  0.975]
----------------------------------------------------------------------
Intercept              21.2177   7.0104  3.0266 0.0025  7.4630 34.9725
C(sex)[T.1]             1.0870   0.7426  1.4639 0.1435 -0.3699  2.5440
C(race)[T.1]            0.2320   0.8435  0.2750 0.7833 -1.4230  1.8869
C(education)[T.2]       0

In [163]:
p_den = fit_model.predict()

In [164]:
fit_model.scale

102.58983402478587

In [165]:
from scipy.stats import norm

In [166]:
density_d = norm.pdf(nhefs_nmv_s.smkintensity82_71, p_den, fit_model.scale**.5)

In [167]:
fit_model = sm.OLS.from_formula('smkintensity82_71 ~ 1', nhefs_nmv_s).fit()

print fit_model.summary2()

                  Results: Ordinary least squares
Model:              OLS               Adj. R-squared:     0.000    
Dependent Variable: smkintensity82_71 AIC:                8756.0778
Date:               2018-01-20 15:53  BIC:                8761.1357
No. Observations:   1162              Log-Likelihood:     -4377.0  
Df Model:           0                 F-statistic:        inf      
Df Residuals:       1161              Prob (F-statistic): nan      
R-squared:          0.000             Scale:              109.58   
---------------------------------------------------------------------
             Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
---------------------------------------------------------------------
Intercept   -2.0577     0.3071   -6.7007   0.0000   -2.6602   -1.4552
-------------------------------------------------------------------
Omnibus:              40.502        Durbin-Watson:           1.993 
Prob(Omnibus):        0.000         Jarque-Bera (JB):     

In [168]:
p_num = fit_model.predict()

In [169]:
density_n = norm.pdf(nhefs_nmv_s.smkintensity82_71, p_num, fit_model.scale**.5)

In [170]:
nhefs_sw_a = density_n / density_d

In [171]:
gee = sm.GEE.from_formula('wt82_71 ~ smkintensity82_71 + I(smkintensity82_71 ** 2)', groups='seqn', data=nhefs_nmv_s, weights=nhefs_sw_a).fit()
print gee.summary()

                               GEE Regression Results                              
Dep. Variable:                     wt82_71   No. Observations:                 1162
Model:                                 GEE   No. clusters:                     1162
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                           Gaussian   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 20 Jan 2018   Scale:                          60.627
Covariance type:                    robust   Time:                         15:54:03
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     2.0045      0.295      6.7

# 12.5

In [173]:
pd.pivot_table(nhefs_nmv, ['death'], columns=['qsmk'])

qsmk,0,1
death,0.171969,0.225806


In [195]:
from statsmodels.api import families

gee = sm.GEE.from_formula('death ~ qsmk', groups='seqn', data=nhefs_nmv_s, family=families.Binomial(), weights=nhefs_sw_a).fit()
print gee.summary()

                               GEE Regression Results                              
Dep. Variable:                       death   No. Observations:                 1162
Model:                                 GEE   No. clusters:                     1162
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                           Binomial   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     5
Date:                     Sat, 20 Jan 2018   Scale:                           1.000
Covariance type:                    robust   Time:                         16:13:32
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5026      0.098    -15.323      0.000      -1.695      -1.310
qsmk   

# 12.6

In [201]:
nhefs_nmv[['sex']].describe()

,sex
count,1566.00000
mean,0.51341
std,0.49998
min,0.00000
25%,0.00000
50%,1.00000
75%,1.00000
max,1.00000


In [202]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))


m = ModelDesc([LookupFactor('qsmk')], model)
m.describe()

'qsmk ~ C(sex) + C(race) + age + I(age ** 2) + C(education) + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + C(exercise) + C(active) + wt71 + I(wt71 ** 2)'

In [203]:
fit_model = sm.Logit.from_formula(m.describe(), nhefs_nmv).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.535408
         Iterations 6
                            Results: Logit
Model:                Logit              No. Iterations:     6.0000   
Dependent Variable:   qsmk               Pseudo R-squared:   0.061    
Date:                 2018-01-20 16:17   AIC:                1714.8968
No. Observations:     1566               BIC:                1816.6661
Df Model:             18                 Log-Likelihood:     -838.45  
Df Residuals:         1547               LL-Null:            -893.03  
Converged:            1.0000             Scale:              1.0000   
----------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------
Intercept              -2.2425   1.3808 -1.6240 0.1044 -4.9489  0.4639
C(sex)[T.1]            -0.5275   0.1540 -3.4241 0.0006 -0.8294 -0.2255
C(r

In [204]:
pd_qsmk = fit_model.predict()

In [205]:
fit_model = sm.Logit.from_formula('qsmk ~ sex', nhefs_nmv).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.567819
         Iterations 5
                         Results: Logit
Model:              Logit            No. Iterations:   5.0000   
Dependent Variable: qsmk             Pseudo R-squared: 0.004    
Date:               2018-01-20 16:18 AIC:              1782.4086
No. Observations:   1566             BIC:              1793.1211
Df Model:           1                Log-Likelihood:   -889.20  
Df Residuals:       1564             LL-Null:          -893.03  
Converged:          1.0000           Scale:            1.0000   
-----------------------------------------------------------------
             Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
-----------------------------------------------------------------
Intercept   -0.9016    0.0799  -11.2789  0.0000  -1.0583  -0.7450
sex         -0.3202    0.1160   -2.7595  0.0058  -0.5476  -0.0928



In [206]:
pn_qsmk = fit_model.predict()

In [207]:
pd_qsmk[nhefs_nmv.qsmk == False] = 1 - pd_qsmk[nhefs_nmv.qsmk == False]
pn_qsmk[nhefs_nmv.qsmk == False] = 1 - pn_qsmk[nhefs_nmv.qsmk == False]

nhefs_sw = pn_qsmk / pd_qsmk

In [208]:
gee = sm.GEE.from_formula('wt82_71 ~ qsmk*sex', groups='seqn', data=nhefs_nmv, weights=nhefs_sw).fit()
print gee.summary()

                               GEE Regression Results                              
Dep. Variable:                     wt82_71   No. Observations:                 1566
Model:                                 GEE   No. clusters:                     1566
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                           Gaussian   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 20 Jan 2018   Scale:                          60.953
Covariance type:                    robust   Time:                         16:21:21
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7844      0.310      5.759      0.000       1.177       2.392
qsmk   

# 12.7

In [303]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))


m = ModelDesc([LookupFactor('qsmk')], model)
m.describe()

'qsmk ~ C(sex) + C(race) + age + I(age ** 2) + C(education) + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + C(exercise) + C(active) + wt71 + I(wt71 ** 2)'

In [304]:
fit_model = sm.Logit.from_formula(m.describe(), nhefs).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.542264
         Iterations 6
                            Results: Logit
Model:                Logit              No. Iterations:     6.0000   
Dependent Variable:   qsmk               Pseudo R-squared:   0.058    
Date:                 2018-01-20 17:00   AIC:                1804.6975
No. Observations:     1629               BIC:                1907.2162
Df Model:             18                 Log-Likelihood:     -883.35  
Df Residuals:         1610               LL-Null:            -938.14  
Converged:            1.0000             Scale:              1.0000   
----------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------
Intercept              -1.9889   1.2413 -1.6023 0.1091 -4.4218  0.4440
C(sex)[T.1]            -0.5075   0.1482 -3.4238 0.0006 -0.7981 -0.2170
C(r

In [305]:
pd_qsmk = fit_model.predict()

In [306]:
fit_model = sm.Logit.from_formula('qsmk ~ 1', nhefs).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.575901
         Iterations 5
                         Results: Logit
Model:              Logit            No. Iterations:   5.0000   
Dependent Variable: qsmk             Pseudo R-squared: 0.000    
Date:               2018-01-20 17:00 AIC:              1878.2862
No. Observations:   1629             BIC:              1883.6819
Df Model:           0                Log-Likelihood:   -938.14  
Df Residuals:       1628             LL-Null:          -938.14  
Converged:          1.0000           Scale:            1.0000   
-----------------------------------------------------------------
             Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
-----------------------------------------------------------------
Intercept   -1.0318    0.0563  -18.3283  0.0000  -1.1421  -0.9215



In [307]:
pn_qsmk = fit_model.predict()

In [308]:
from patsy import EvalFactor, ModelDesc, Term, LookupFactor

model = [Term([])]
model.append(LookupFactor('C(sex)'))
model.append(LookupFactor('C(race)'))
model.append(LookupFactor('age'))
model.append(EvalFactor('I(age**2)'))
model.append(LookupFactor('C(education)'))
model.append(LookupFactor('smokeintensity'))
model.append(EvalFactor('I(smokeintensity**2)'))
model.append(LookupFactor('smokeyrs'))
model.append(EvalFactor('I(smokeyrs**2)'))
model.append(LookupFactor('C(exercise)'))
model.append(LookupFactor('C(active)'))
model.append(LookupFactor('wt71'))
model.append(EvalFactor('I(wt71**2)'))

model.append(LookupFactor('qsmk'))


m = ModelDesc([LookupFactor('cens')], model)
m.describe()

'cens ~ C(sex) + C(race) + age + I(age ** 2) + C(education) + smokeintensity + I(smokeintensity ** 2) + smokeyrs + I(smokeyrs ** 2) + C(exercise) + C(active) + wt71 + I(wt71 ** 2) + qsmk'

In [309]:
nhefs.cens = nhefs.cens.astype('int')

fit_model = sm.Logit.from_formula(m.describe(), nhefs).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.142836
         Iterations 8
                            Results: Logit
Model:                 Logit              No. Iterations:     8.0000  
Dependent Variable:    cens               Pseudo R-squared:   0.127   
Date:                  2018-01-20 17:00   AIC:                505.3600
No. Observations:      1629               BIC:                613.2744
Df Model:              19                 Log-Likelihood:     -232.68 
Df Residuals:          1609               LL-Null:            -266.68 
Converged:             1.0000             Scale:              1.0000  
----------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------
Intercept               4.0145   2.5761  1.5583 0.1192 -1.0346  9.0635
C(sex)[T.1]             0.0573   0.3303  0.1735 0.8622 -0.5900  0.7046
C(r

In [310]:
pd_cens = fit_model.predict()

In [311]:
fit_model = sm.Logit.from_formula('cens ~ qsmk + 1', nhefs).fit()

print fit_model.summary2()

Optimization terminated successfully.
         Current function value: 0.161989
         Iterations 7
                        Results: Logit
Model:              Logit            No. Iterations:   7.0000  
Dependent Variable: cens             Pseudo R-squared: 0.010   
Date:               2018-01-20 17:00 AIC:              531.7602
No. Observations:   1629             BIC:              542.5516
Df Model:           1                Log-Likelihood:   -263.88 
Df Residuals:       1627             LL-Null:          -266.68 
Converged:          1.0000           Scale:            1.0000  
----------------------------------------------------------------
            Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
----------------------------------------------------------------
Intercept  -3.4212    0.1649  -20.7532  0.0000  -3.7443  -3.0981
qsmk        0.6411    0.2639    2.4291  0.0151   0.1238   1.1584



In [312]:
pn_cens = fit_model.predict()

In [313]:
pd_qsmk

array([0.10899818, 0.14888633, 0.13419533, ..., 0.13818249, 0.44782907,
       0.17771579])

In [314]:
is_cens = nhefs.cens != 1

pd_qsmk[nhefs.qsmk == False] = 1 - pd_qsmk[nhefs.qsmk == False]
pn_qsmk[nhefs.qsmk == False] = 1 - pn_qsmk[nhefs.qsmk == False]

pd_qsmk = pd_qsmk[is_cens]
pn_qsmk = pn_qsmk[is_cens]
pd_cens = pd_cens[is_cens]
pn_cens = pn_cens[is_cens]

qsmk_ratio = pn_qsmk / pd_qsmk
sw = qsmk_ratio * (1 - pn_cens) / (1 - pd_cens)

In [315]:
qsmk_ratio

array([0.82745299, 0.86623226, 0.85153401, ..., 0.86360829, 1.33520633,
       1.47841607])

In [316]:
sw

array([0.82268622, 0.86329518, 0.83918764, ..., 0.8770074 , 1.40724683,
       1.42001644])

In [317]:
gee = sm.GEE.from_formula('wt82_71 ~ qsmk', groups='seqn', data=nhefs_nmv, weights=sw).fit()
print gee.summary()

                               GEE Regression Results                              
Dep. Variable:                     wt82_71   No. Observations:                 1566
Model:                                 GEE   No. clusters:                     1566
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   1
Family:                           Gaussian   Mean cluster size:                 1.0
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 20 Jan 2018   Scale:                          61.861
Covariance type:                    robust   Time:                         17:00:57
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6620      0.233      7.141      0.000       1.206       2.118
qsmk   